**Flavia Penta - Giovanni Giunta - Luca Palluzzi**


# Homework 3 - Which book would you recomend?

**Goal of the homework**: Build a search engine over the "best books ever" list of [GoodReads](https://www.goodreads.com/). Unless differently specified, all the functions must be implemented from scratch.


## 1. Data collection

For this homework, there is no provided dataset, but you have to build your own. Your search engine will run on text documents. So, here
we detail the procedure to follow for the data collection. 

*import*

In [ ]:
import pandas as pd
from collections import defaultdict
from collections import Counter
from pathlib import Path
from bs4 import BeautifulSoup
import requests
import csv 
import re
from langdetect import detect
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer 
from nltk.tokenize import RegexpTokenizer
import json
import nltk
import re
import heapq
import math
nltk.download('stopwords')

### 1.1. Get the list of books

We start from the list of books to include in your corpus of documents. In particular, we focus on the [best books ever list](https://www.goodreads.com/list/show/1.Best_Books_Ever?page=1). From this list we want to **collect the url** associated to each book in the list.
As you realize, the list is long and splitted in many pages. We ask you to retrieve only the urls of the books listed in the first 300 pages.

The output of this step is a `.txt` file whose single line corresponds to a book's url.

In [ ]:
f = open("homework.txt", "w")

for i in range(1,301):
    page = requests.get("https://www.goodreads.com/list/show/1.Best_Books_Ever?page=" + str(i))
    soup = BeautifulSoup(page.content, features="lxml")
    links = soup.find_all('a', itemprop='url', class_='bookTitle')
    for link in links:
        fullLink = link.get('href')
        f.write('https://www.goodreads.com' + fullLink + '\n')
        
f.close()

### 1.2. Crawl books

Once you get all the urls in the first 300 pages of the list, you:

1. Download the html corresponding to each of the collected urls.
2. After you collect a single page, immediatly save its `html` in a file. In this way, if your program stops, for any reason, you will not loose the data collected up to the stopping point. More details in **Important (2)**. 
3. Organize the entire set of downloaded `html` pages into folders. Each folder will contain the `htmls` of the books in page 1, page 2, ... of the list of books.

#### Important
*Due to the large amount of pages you need to download, we give you the following tipds that help you to speed up some time-consuming operations.*

1. **[Save time downloading files]** You are asked to crawl tons of pages, and this will take a lot of time. To speed up the operation, we suggest you to work in parallel with your group's colleagues. In particular, using the same code, each component of the group can be in charge of downloading a subset of pages (e.g., the first 100). **PAY ATTENTION:** Once obtained all the pages, merge your results in a unique dataset. In fact, the search engine must look up for results in the whole set of documents.

2. **[Save your data]** It is not nice to restart a crawling procedure, given its runtime. For this reason, it is extremely important that for every time you crawl a page, you **must** save it with the name `article_i.html`, where `i` corresponds to the number of articles you have already downloaded. In such way, if something goes bad, you can restart your crawling procedure from the *i+1*-th document.

In [ ]:
path = './HW3/st/'

f = open("homework.txt", "r")

ff = f.readlines()

for i in range(30000, len(ff)+1):
    
    folderName = "folder-" + str(i) + "/"
    fileName = "article_" + str(i) + ".html"
    

    url = ff[i-1]
    
    Path(path + folderName).mkdir(parents=True, exist_ok=True)

    page = requests.get(url)
    code = str(page.text)

    with open(path + folderName + fileName, "w", encoding="utf-8") as z:
        z.write(code)

    z.close()
f.close()

### 1.3 Parse downloaded pages

At this point, you should have all the html documents about the books of interest and you can start to extract the books informations. The list of information we desire for each book are the following:

1. Title (to save as `bookTitle`)
2. Series (to save as `bookSeries`)
3. Author(s), the first box in the picture below (to save as `bookAuthors`)
4. Ratings, average stars (to save as `ratingValue`)
5. Number of givent ratings (to save as `ratingCount`)
6. Number of reviews (to save as `reviewCount`)
7. The entire plot (to save as `Plot`)
8. Number of pages (to save as `NumberofPages`)
9. Published (Publishing Date)
10. Characters
11. Setting
12. Url


For each book, you create a `article_i.tsv` file of this structure:

```
bookTitle \t bookSeries \t  ... \t Setting
```

If an info it is not present, you just leave it as an empty string.


__Example__:
  
```
bookTitle \t bookSeries \t  ... \t Setting
    
The Hunger Games \t The Hunger Games \t ... \t District 12, Panem, Capitol, Panem, Panem (United States)
```

#### Important

It may happen that the plot is not in English (e.g., [The Pastures of Heaven](https://www.goodreads.com/book/show/186369.The_Pastures_of_Heaven)). So, once you get the plot of a book, check if it is written in english. To do it, we suggest you to use [langdetect](https://pypi.org/project/langdetect/). If the language is **not** English, just discard the book.



In [1]:
personalPath = "./HW3/st/folder-"

data = ['bookTitle', 'bookSeries', 'bookAuthors', 'ratingValue', "ratingCount",\
        "reviewCount", "plot", "numberOfPages", "publishingDate", "characters", "setting", "url"]

personalPath = "D:/Storage file PC/Documenti/Università/Data Science/Anno 1/Semestre 1/Algorthmic Methods of Data Mining/Homeworks/HW3/st/folder-"

for i in range(18001, 30001):
    with open(personalPath + str(i) + "/article_" + str(i) + ".html", 'rb') as html: 
        soup = BeautifulSoup(html,"html.parser")
    #plot = soup.find('div',id='description').text.strip()  
    try:
        plot = soup.find('div',id='description').text.strip()
        if detect(plot)=='en':
            lista=[]

            #title
            try:
                lista.append(soup.find('h1').text.strip())
            except:
                lista.append('')

                #bookseries
            try:
                lista.append(soup.find('h2',id='bookSeries').text.strip())
            except:
                lista.append('')

                #author name
            try:
                lista.append(soup.find('a',class_='authorName').text.strip())
            except:
                lista.append('')

                #rating value
            try:
                lista.append(soup.find('span', itemprop='ratingValue').text.strip())
            except:
                lista.append('')

                #ratingCount
            try:
                lista.append(soup.find_all('a',class_='gr-hyperlink',href='#other_reviews')[0]\
                             .text.strip().replace('\r', '').replace('\n', '').split()[0])
            except:
                lista.append('')

                #reviewCount
            try:
                lista.append(soup.find_all('a',class_='gr-hyperlink',href='#other_reviews')[1]\
                             .text.strip().replace('\r', '').replace('\n', '').split()[0])
            except:
                lista.append('')

                #plot
            try:
                if plot[-7:] == '...more':
                    lista.append(soup.find('div',id='description').contents[3].text)
                else:
                    lista.append(plot)
            except:
                lista.append('')

                #number of pages
            try:
                lista.append(soup.find('span', itemprop='numberOfPages').text.strip().split()[0])
            except:
                lista.append('')

                #Publishing Date
            try:
                a=soup.find_all('div', class_='row')[1].text
                match_obj = re.split('Published', re.split('by', a)[0])[1]
                lista.append(match_obj.strip())
            except:
                lista.append('')

                #characters
            try:
                l1=[]
                for d in soup.find_all('a',href=re.compile(r'/characters/*')):
                    l1.append(d.text)
                    s1=",".join(l1)
                lista.append(s1)
            except:
                lista.append('')

                #setting
            try:
                l2=[]
                for e in soup.find_all('a',href=re.compile(r'/places/*')):
                    l2.append(e.text)
                    s2=",".join(l2)
                lista.append(s2)
            except:
                lista.append('')

                #URL
            lista.append(soup.find('link')['href'].strip())

            path = personalPath + str(i) + '/article_' + str(i)+ '.tsv'

            with open(path, 'w', newline='',encoding="utf-8") as f_output:
                tsv_output = csv.writer(f_output, delimiter='\t')
                tsv_output.writerow(data)
                tsv_output.writerow(lista)
                f_output.close()
        
        else:
            print('This book is not in english: '+ str(i))
            
    except:
        print('Missing plot for book: '+ str(i))

In [ ]:
i = 1
index = 1

new_file = open('index_books.tsv', 'w')

while i <= 30000:
    try:
        art_f = open(personalPath + str(i) + "/article_" + str(i) + ".tsv", 'r')
        art = art_f.readlines()[1]
        new_file.write(str(index) + "\t" + art)
        art_f.close()
        i += 1
        index += 1
    except:
        i += 1

new_file.close()

## 2. Search Engine

Now, we want to create two different Search Engines that, given as input a query, return the books that match the query.

First, you must pre-process all the information collected for each book by

1. Removing stopwords
2. Removing punctuation
3. Stemming
4. Anything else you think it's needed

For this purpose, you can use the [nltk library](https://www.nltk.org/).

### 2.1. Conjunctive query
For the first version of the search engine, we narrow our interest on the `Plot` of each document. It means that you will evaluate queries only with respect to the book's plot.

#### 2.1.1) Create your index!

Before building the index, 
* Create a file named `vocabulary`, in the format you prefer, that maps each word to an integer (`term_id`).

Then, the first brick of your homework is to create the Inverted Index. It will be a dictionary of this format:

```
{
term_id_1:[document_1, document_2, document_4],
term_id_2:[document_1, document_3, document_5, document_6],
...}
```
where _document\_i_ is the *id* of a document that contains the word.

__Hint:__ Since you do not want to compute the inverted index every time you use the Search Engine, it is worth to think to store it in a separate file and load it in memory when needed.

In [ ]:
nodigit = lambda wordslist : [word for word in wordslist if word.isalpha()]

f = open("index_books.tsv", 'r')
books = f.readlines()

new_file = open('vocabolary.tsv', 'w')

ps = PorterStemmer()

term_id = 1
document_id = 1

vocabolary = dict()
diz = defaultdict(set)

for book in books:
    tokenizer = RegexpTokenizer(r"[a-zA-Z]+") 
    text_tokens = nodigit(tokenizer.tokenize(book.split('\t')[7]))
    tokens_without_sw = {word for word in text_tokens if not word in stopwords.words()}
    for word in tokens_without_sw:
        w = ps.stem(word.lower())
        if w not in vocabolary:
            vocabolary[w] = term_id
            diz[term_id].add(document_id)
            new_file.write(w + "\t" + str(term_id) + '\n')
            term_id += 1
        else:
            diz[vocabolary[w]].add(document_id)
    print('Finished document ' + str(document_id))
    document_id += 1
    
new_file.close()
f.close()


with open("dictionary.json", "w") as outfile: 
    json.dump(dict(zip(diz.keys(), map(list, diz.values()))), outfile, indent = 4)

#### 2.1.2) Execute the query
Given a query, that you let the user enter:

```
survival games
```

the Search Engine is supposed to return a list of documents.

##### What documents do we want?
Since we are dealing with conjunctive queries (AND), each of the returned documents should contain all the words in the query.
The final output of the query must return, if present, the following information for each of the selected documents:

* `bookTitle`
* `Plot`
* `Url`

__Example Output__:

| bookTitle | Plot | Url |
|:-----------------------------:|:-----:|:------------------------------------------------------------:|
| The Hunger Games | ... | https://www.goodreads.com/book/show/2767052-the-hunger-games |
| Harry Potter and the Goblet of Fire | ... | https://www.goodreads.com/book/show/6.Harry_Potter_and_the_Goblet_of_Fire |
| Catching Fire | ... | https://www.goodreads.com/book/show/6148028-catching-fire |

If everything works well in this step, you can go to the next point, and make your Search Engine more complex and better in answering queries.

In [10]:
ds = pd.read_csv('index_books.tsv', header=None, sep='\t', usecols=[0,1,7,12])

ds.rename(columns={0:'index', 1:'bookTitle', 7:'plot', 12:'url'}, inplace=True)

voc = dict()
with open('vocabolary.tsv') as f:
    for col1, col2 in csv.reader(f, delimiter='\t'):
        voc[col1] = col2
        
with open('dictionary.json') as f:
    dt = json.load(f) # dictionary


def query(q):

    ps = PorterStemmer()

    q = q.strip().split() # input from user
    
    ps = PorterStemmer()
    q = [ps.stem(w).lower() for w in q]

    # elaborate query
    
    # take term_id(s)
    term = list()
    for w in q:
        try:
            term.append(voc[w])
        except:
            pass
    # matching documents
    if len(term):
        doc = set(dt[term[0]])
        for i in range(1, len(term)):
            doc = doc.intersection(dt[term[i]])
        # take row from books
        return ds[ds['index'].isin(list(doc))].head()
    else:
        return "There aren't documents for each word of this query"

In [5]:
# the excution

query('survival games')

,index,bookTitle,plot,url
0,1,The Hunger Games,"Could you survive on your own in the wild, wit...",https://www.goodreads.com/book/show/2767052-th...
196,204,Catching Fire,SPARKS ARE IGNITING.FLAMES ARE SPREADING.AND T...,https://www.goodreads.com/book/show/6148028-ca...
282,295,Mockingjay,The final book in the ground-breaking HUNGER G...,https://www.goodreads.com/book/show/7260188-mo...
300,313,Legend,What was once the western United States is now...,https://www.goodreads.com/book/show/9275658-le...
576,611,The Magus,"This daring literary thriller, rich with eroti...",https://www.goodreads.com/book/show/16286.The_...



### 2.2) Conjunctive query & Ranking score

For the second search engine, given a query, we want to get the *top-k* (the choice of *k* it's up to you!) documents related to the query. In particular:

* Find all the documents that contains all the words in the query.
* Sort them by their similarity with the query
* Return in output *k* documents, or all the documents with non-zero similarity with the query when the results are less than _k_. You __must__ use a heap data structure (you can use Python libraries) for maintaining the *top-k* documents.

To solve this task, you will have to use the *tfIdf* score, and the _Cosine similarity_. The fielf to consider it is still the plot. Let's see how.


#### 2.2.1) Inverted index
Your second Inverted Index must be of this format:

```
{
term_id_1:[(document1, tfIdf_{term,document1}), (document2, tfIdf_{term,document2}), (document4, tfIdf_{term,document4}), ...],
term_id_2:[(document1, tfIdf_{term,document1}), (document3, tfIdf_{term,document3}), (document5, tfIdf_{term,document5}), (document6, tfIdf_{term,document6}), ...],
...}
```

Practically, for each word you want the list of documents in which it is contained in, and the relative *tfIdf* score.

__Tip__: *tfIdf* values are invariant with respect to the query, for this reason you can precalculate them.

In [7]:
voc = dict()
with open('vocabolary.tsv') as f:
    for col1, col2 in csv.reader(f, delimiter='\t'):
        voc[col1] = col2
        
with open('dictionary.json') as f:
    dt = json.load(f) # dictionary
    
ds = dict()
with open('index_books.tsv') as f:
    for row in csv.reader(f, delimiter='\t'):
        if len(row) == 13:
            ds[row[0]] = row[7]


result = defaultdict(list)
term_idf = defaultdict(float)

for doc_id in ds:
    
    ps = PorterStemmer()
    tokenizer = RegexpTokenizer(r"[a-zA-Z]+") 
    text_tokens = nodigit(tokenizer.tokenize(ds[doc_id]))
    tokens_without_sw = [ps.stem(w.lower()) for w in text_tokens if not w in stopwords.words()]
    
    plotLength = len(tokens_without_sw)
    count = Counter(tokens_without_sw)
    
    for word in count:
        freq = count[word]
        try:
            term_id = str(voc[word])
            idf = 1.0 + math.log( float(len(ds)) / len( dt[term_id] ) )
            tf = freq / plotLength
            tfIdf = tf * idf

            
            heapq.heappush(result[term_id], (tfIdf, doc_id))
            term_idf[term_id] = idf

        except:
            pass

inv_ind = defaultdict(list)
for term, tup_list in result.items():
    for tup in tup_list:
        inv_ind[term].append( (int(tup[1]), tup[0]) )


with open("inverted_index.json", "w") as outfile: 
    json.dump(result, outfile, indent = 4)

with open("term_idf.json", "w") as outfile: 
    json.dump(term_idf, outfile, indent = 4)

#### 2.2.2) Execute the query

In this new setting, given a query you get the right set of documents (i.e., those containing all the words in the query) and sort them according to their similairty to the query. For this purpose, as scoring function we will use the Cosine Similarity with respect to the *tfIdf* representations of the documents.

Given a query, that you let the user enter:
```
survival games
```
the search engine is supposed to return a list of documents, __ranked__ by their Cosine Similarity with respect to the query entered in input.

More precisely, the output must contain:
* `bookTitle`
* `Plot`
* `Url`
* The similarity score of the documents with respect to the query


__Example Output__:

| bookTitle | Plot | Url | Similarity |
|:-------------------------------------:|:-----:|:---------------------------------------------------------------------:|------------|
| The Hunger Games | ... | https://www.goodreads.com/book/show/2767052-the-hunger-games | 0.96 |
| Harry Potter and the Goblet of Fire | ... | https://www.goodreads.com/book/show/6.Harry_Potter_and_the_Goblet_of_Fire | 0.92 |
| Catching Fire | ... | https://www.goodreads.com/book/show/6148028-catching-fire | 0.87 |



In [11]:
with open('term_idf.json') as f:
    term_idf = json.load(f)

with open('inverted_index2.json') as f:
    inverted = json.load(f)
    
inv_ind = defaultdict(dict)
for term in inverted:
    for t in inverted[term]:
        inv_ind[term][t[0]] = t[1]

dot = lambda x, y : sum(xi*yi for xi, yi in zip(x, y))
square = lambda x : [v**2 for v in x]
det = lambda x : math.sqrt(sum(square(x)))
    
def similarity(q):
    ps = PorterStemmer()
    # execute query
    err = "There aren't documents for each word of this query"
    q_result = query(q)
    if not isinstance(q_result, str):
        q = q.strip().split() # input from user
        q = [ps.stem(w).lower() for w in q]
        # create a list of ifidf of terms
        term_ifidf = list()
        tf = 1/len(q)
        for w in q:
            term_ifidf += [term_idf[voc[w]]*tf]
        # create a list of ifidf of document
        doc_ifidf = defaultdict(list)
        for d_id in q_result['index']:
            for w in q:
                doc_ifidf[d_id].append(inv_ind[voc[w]][d_id])
        #compare value and calculate similarity
        cos_sim = list()
        det_q = det(term_ifidf)
        for doc in q_result['index']:
            prod = dot(doc_ifidf[doc], term_ifidf)
            det_doc = det(doc_ifidf[doc])
            cos_sim += [(prod / (det_q * det_doc))]
        q_result['similarity'] = cos_sim
        return q_result.sort_values(by=['similarity', 'index'], ascending=False).head()
    else:
        return err


In [12]:
# the excution of a query

similarity('survival games')

,index,bookTitle,plot,url,similarity
300,313,Legend,What was once the western United States is now...,https://www.goodreads.com/book/show/9275658-le...,1.000000
576,611,The Magus,"This daring literary thriller, rich with eroti...",https://www.goodreads.com/book/show/16286.The_...,1.000000
196,204,Catching Fire,SPARKS ARE IGNITING.FLAMES ARE SPREADING.AND T...,https://www.goodreads.com/book/show/6148028-ca...,1.000000
0,1,The Hunger Games,"Could you survive on your own in the wild, wit...",https://www.goodreads.com/book/show/2767052-th...,0.979797
282,295,Mockingjay,The final book in the ground-breaking HUNGER G...,https://www.goodreads.com/book/show/7260188-mo...,0.956506
